In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
from mediapipe.python.solutions import face_mesh as mp_face_mesh

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(landmarks, w, h):
    top = np.array([landmarks[13].x * w, landmarks[13].y * h])
    bottom = np.array([landmarks[14].x * w, landmarks[14].y * h])
    left = np.array([landmarks[78].x * w, landmarks[78].y * h])
    right = np.array([landmarks[308].x * w, landmarks[308].y * h])
    return dist.euclidean(top, bottom) / dist.euclidean(left, right)

# Inicializar Face Mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]

EAR_THRESHOLD = 0.22
EAR_FRAMES_REQUIRED = 2
MAR_THRESHOLD = 0.5

blink_counter = 0
tears = []
fire_particles = []

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    if result.multi_face_landmarks:
        face = result.multi_face_landmarks[0]
        landmarks = face.landmark

        # Ojos
        left_eye = np.array([(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in LEFT_EYE_IDX])
        right_eye = np.array([(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in RIGHT_EYE_IDX])
        left_EAR = eye_aspect_ratio(left_eye)
        right_EAR = eye_aspect_ratio(right_eye)
        EAR = (left_EAR + right_EAR) / 2

        if EAR < EAR_THRESHOLD:
            blink_counter += 1
        else:
            if blink_counter >= EAR_FRAMES_REQUIRED:
                lx, ly = int(np.mean(left_eye[:,0])), int(np.mean(left_eye[:,1])) + 15
                rx, ry = int(np.mean(right_eye[:,0])), int(np.mean(right_eye[:,1])) + 15
                tears.append([lx, ly])
                tears.append([rx, ry])
            blink_counter = 0

        MAR = mouth_aspect_ratio(landmarks, w, h)
        if MAR > MAR_THRESHOLD:
            # Obtener puntos de la boca para una base más amplia
            mouth_left = int(landmarks[78].x * w)
            mouth_right = int(landmarks[308].x * w)
            mouth_top = int((landmarks[13].y + landmarks[14].y)/2 * h)
            mouth_width = mouth_right - mouth_left
            
            for _ in range(8):  # Más partículas para un efecto más denso
                x = mouth_left + np.random.randint(0, mouth_width)
                y = mouth_top + np.random.randint(-5, 5)
                
                vx = np.random.uniform(-0.5, 0.5)
                vy = np.random.uniform(-3, -1.5)  # Más rápido hacia arriba
                
                size = np.random.randint(5, 12)
                life = np.random.randint(15, 40)
                
                fire_particles.append({
                    "pos": [x, y],
                    "vel": [vx, vy],
                    "size": size,
                    "life": life,
                    "max_life": life
                })

    #fuego
    new_particles = []
    for p in fire_particles:
        x, y = p["pos"]
        vx, vy = p["vel"]
        
        # Aplicar gravedad y turbulencia
        vy += 0.1  # Gravedad suave
        vx += np.random.uniform(-0.1, 0.1)  # Turbulencia
        
        # Actualizar posición
        x += vx
        y += vy
        p["pos"] = [x, y]
        p["life"] -= 1
        
        life_ratio = p["life"] / p["max_life"]
        if life_ratio > 0.7:
            # Base - rojo intenso
            color = (0, 0, 255)
        elif life_ratio > 0.4:
            # Medio - naranja
            color = (0, 165, 255)
        else:
            # Punta - amarillo
            color = (0, 255, 255)
            
        # Dibujar partícula con gradiente de tamaño
        if p["life"] > 0:
            # Partícula principal
            cv2.circle(frame, (int(x), int(y)), p["size"], color, -1)
            
            # Añadir brillo interior para efecto de llama
            inner_size = max(1, int(p["size"] * 0.6))
            cv2.circle(frame, (int(x), int(y)), inner_size, (0, 255, 255), -1)
            
            new_particles.append(p)
    fire_particles = new_particles

    # Lágrimas
    new_tears = []
    for (x, y) in tears:
        cv2.circle(frame, (x, y), 10, (255, 200, 100), -1)
        y += 6
        if y < h - 10:
            new_tears.append([x, y])
    tears = new_tears

    cv2.imshow("Lagrimas + Llamas ", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()